# Model Inference Utilities
> Functions that are used to markup up files with a learner

In [1]:
#hide
# default_exp inference_utils
from nbdev.showdoc import *
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Imports

In [2]:
# export
from cellsegment.core import *
from fastai.vision import *
from fastai.utils.mem import *

import cv2
from skimage.measure import label, regionprops

# from skimage import filters
# from skimage.morphology import erosion, dilation, opening, closing, disk
# from scipy.spatial import distance
# import pandas as pd
from pandas import DataFrame

## General Inference Functions

In [ ]:
# export
def run_inferences(learn:Learner, fnames:list, start:int=0, number_files=20) -> list:
    """
    run inferences over a list of `fnames` using `learner`
    - return a list of predictions
    """
    preds = []
    if number_files == 'all':
        __number_files = len(fnames)
    elif isinstance(number_files, int) :
        __number_files = number_files
    else:
        __number_files = 0
    for i in range(start, start+__number_files):
        if i >= len(fnames): break
        img = open_image(fnames[i])
        y, pred, raw_pred = learn.predict(img)
        preds.append({"filename": fnames[i].name, "y": y,"pred": pred,"raw_pred": raw_pred})
    return preds

In [ ]:
# export
def plot_inferences(preds:List, df:DataFrame, src_path:Path=Path(''), label=None, start:int=0, rows:int = 4, cols:int=5) -> None:
    """
    Plot test images with auto markup labels
    """
    fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
    ax_lst = []
    for i, ax in enumerate(axes.flat):
        if i+start >= len(preds): break
        fpath = src_path/preds[i+start]["filename"]
        img = open_image(fpath)
        width, height = img.size
        show_image(img, ax=ax)
        show_image(preds[i+start]["y"], ax=ax,  cmap='tab20', alpha=0.5)
        fn = Path(fpath).name
        # B,L,R,O = df.loc[fn, 'Background'], df.loc[fn, 'Fluke Liver'], df.loc[fn, 'Fluke Rumen'],df.loc[fn, 'Other']
        draw_text(ax, (0, 0), f'{fpath.name}', sz=14, color='red')
        pstr = ''
        if label in df.columns:
            pstr = df.loc[df['Name'] == fn, label].values[0]
            draw_text(ax, (0, height-20), pstr, sz=14, color='red')

        true_label = df.loc[df['Name'] == fn, 'Label'].values[0]
        ax.set_title(f"File {fpath.name} \n GT={true_label} P={pstr}")
        ax_lst.append(ax)

    plt.tight_layout()
    return None

In [3]:
# def 
# def create_probs_df(fnames: List, ) -> DataFrame:
#     """
#     create a data frame of prob from a `fname` list
#      - param fnames: 
#      - return: `DataFrame`
#     """
#     list_filedata = [None] * len(fnames)
#     for i, fn in enumerate(fnames):
#         # img = PIL.Image.open(fn)
#         # img_w, img_h = img.size
#         list_filedata[i] = {'Name': fn.name, 'Background': '', 'Fluke Liver': '', 'Fluke Rumen': '', 'Other': '', 'Pstr': ''}
# 
#     df = pd.DataFrame(list_filedata)
#     df = df[['Name', 'Background', 'Fluke Liver', 'Fluke Rumen', 'Other', 'Pstr']]
#     df.set_index("Name", inplace=True)
#     return df

## Predictions on tiles

In [ ]:
#export
def get_region_props(raw_pred:tensor, layer:int, min_conf:float=0.2, min_area:int = 100) -> list:
    ' return all the region properties of a prediction layer filtered by min confidence and min area '
    arr = raw_pred[layer, :, :].numpy()
    if layer == 0:
        label_arr = label(arr < 1 - min_conf)
    else:
        label_arr = label(arr > min_conf)

    region_props = []
    for region in regionprops(label_arr, arr, cache=True):
        if region.area > min_area:
            cx = int(region.centroid[1])
            cy = int(region.centroid[0])
            region_props.append({
                "class_layer": layer,
                "centroid": region.centroid,
                "mean_intensity": region.mean_intensity,
                "area": region.area,
                "coords": region.coords,
            })
    return region_props

In [ ]:
# export
def calc_probs(raw_pred:tensor)->list:
    """
    Calculate the probabilities from a raw prediction  `raw_pred`
    by finding the average value of all the pixels within a region
    This assumes that there is only one cell in the image
    - `raw_pred`: the raw predictions from the learner
    - `return`:  list of probabilities
    """
    def __get_bool_mask(o):
        min_conf = 0.5
        arr = o[0, :, :].numpy()
        bgnd = get_region_props(o, 0, min_conf)
        if len(bgnd) > 0:
            mask = np.zeros((arr.shape), dtype=bool)
            AAcoords = bgnd[0]["coords"]  # TODO uses first region only
            mask[tuple(AAcoords.T.tolist())] = True
        else:
            mask = None
        return mask

    mask = __get_bool_mask(raw_pred)
    if mask is not None:
        p = [None] * raw_pred.shape[0]
        for i in range(raw_pred.shape[0]):
            p[i] = raw_pred[i, :, :].numpy()[mask].mean().round(2)
    else:
        p = []
    return p

In [4]:
# export
def preds_to_df(preds:List, col_headings: List,  df:DataFrame):
    """
    add the predictions to the dataframe indexed by fname
    - preds: prediction list
    - col_headings: column headings
    - df: dataframe
    - return:
    """
    for c in col_headings:
        if not c in df.columns:
            df[c] = ''
    assert len(col_headings) == len(preds[0]), f' len(col_headings) != len(preds[0])'

    for i, pred in enumerate(preds):
        fn = Path(pred["filename"]).name
        probs = calc_probs(pred["raw_pred"])
        if len(probs) > 0:
            pstr = f"B:{probs[0]:3.2f} L:{probs[1]:3.2f} R:{probs[2]:3.2f}"
            df.loc[df['Name'] == fn, col_headings] = probs

## Predictions on whole images

In [ ]:
#export
def find_prediction_blobs(img, CONF=0.5, min_area=500,offset=0):
    ' plot the prediction blobs on an image '
    # region props seems to have region.max_intensity errors if no data not np.int
    SCALE = 100
    CONF *= SCALE

    img = (img * (SCALE / img.max())).astype(np.int)
    img[img[:, :, 0] < CONF, 0] = 0
    img[img[:, :, 1] < CONF, 1] = 0

    predictions = []

    fill = (255, 0, 0)
    label_image0 = label(img[:, :, 0] > CONF)
    label_image1 = label(img[:, :, 1] > CONF)

    print ('Label 0')
    img = np.array(img)  # helped with a cv error?
    for region in regionprops(label_image0, img[:, :, 0], cache=True):
        if region.area > min_area:
            cx = int(region.centroid[1]) + offset
            cy = int(region.centroid[0])
            predictions.append({"label": 'Strongyle', "particleType": 1, "point": [cx, cy],
                                "probability": int(region.max_intensity.round(2))})
            print(' 0:', [cx,cy], 'area:',  region.area,
                      'max',  region.max_intensity.round(2),
                      'mean', region.mean_intensity.round(2))
    print ('Label 1')
    fill = (0, 255, 0)
    img = np.array(img)  # helped with a cv error?
    for region in regionprops(label_image1, img[:, :, 1]):
        if region.area > min_area:
            cx = int(region.centroid[1]) + offset
            cy = int(region.centroid[0])
            predictions.append({"label": 'Nematodirus', "particleType": 2, "point": [cx, cy],
                                "probability": int(region.max_intensity.round(2))})
            print(' 0:', [cx, cy], 'area:', region.area,
                  'max', region.max_intensity.round(2),
                  'mean', region.mean_intensity.round(2))

    return predictions, img

In [ ]:
#export
def draw_labels_on_image(img, json, radius=40, offset=0):
    font = cv2.FONT_HERSHEY_SIMPLEX
    for s, sh in enumerate(json['shapes']):
        if sh["label"][:3] == "Str":
            fill = (1, 0, 0)
        elif sh["label"][:3] == "Nem":
            fill = (0, 1, 0)
        else:
            print('[Error]: unknown label')

        draw = 'None'
        if sh['shape_type'] == 'circle':
            draw = 'circle'
            probability = str(sh['probability']) if 'probability' in sh else ''
        elif sh['shape_type'] == 'rectangle':
            draw = 'rectangle'
            probability = str(sh['probability']) if 'probability' in sh else ''
        else:
            print("Unknown shape_type", sh['shape_type'])

        xy = np.asarray(sh["points"])
        ave = np.mean(xy, axis=0)

        cx = int(ave[0]) - offset
        cy = int(ave[1])

        if draw == 'circle':
            cv2.circle(img, (cx, cy), radius, fill, 2)
            cv2.circle(img, (cx, cy), radius, fill, 2)

        elif draw == 'rectangle':
            cv2.rectangle(img, (cx - radius, cy - radius), (cx + radius, cy + radius), fill, 2)
            cv2.putText(img, probability, (int(cx - radius), cy - radius - 5), font, 0.8, fill, 2, cv2.LINE_AA)

    return img

In [ ]:
# export
# def unetCountEggsInWell(learn, img):
#     PAD = 50
#     TM, TN = 2, 2
#     min_area = 100
#     anno_radius = 20
#
#     img = resize(img, (2464 // 2, 3450 // 2))
#
#     offset, img = crop2well(img, width=2464 // 2, plot=False)
#     img_f = img.transpose(2, 0, 1).astype(np.float32)
#     imglist = dice_image(torch.from_numpy(img_f), TM=TM, TN=TN, pad=PAD)
#     print("Number of tiles = ", len(imglist))
#
#     pred_list = []
#     for n, im in enumerate(imglist):
#         print(f"Predictig Tile {n}")
#         pc, pi, o = learn.predict(im)
#         pred_list.append(o)
#
#     pred_img = tile_image(pred_list, TM=TM, TN=TN, pad=PAD)
#
#     pred_img[0, :, :] = pred_img[1, :, :]
#     pred_img[1, :, :] = pred_img[2, :, :]
#     pred_img[2, :, :] = 0
#
#     pred_img[pred_img<0.3] = 0
#     pred_im

## Dataframe functions

In [5]:
# export
def add_cols_to_df(df: DataFrame, col_list: List) -> DataFrame:
    """
    To dataframe `df` add columns in `col_list`
    - df: dataframe to add columns to
    - col_list: list of columns to add
    - return: the df
    """
    add_col = [''] * df.shape[0]
    for l in col_list:
        df[l] = add_col
    return df

##Drawing functions
> Draw text and graphics on an matplotlib image

In [6]:
# export
def bb_hw(a): return np.array([a[1], a[0], a[3] - a[1], a[2] - a[0]])

def draw_outline(o, lw):
    o.set_path_effects([patheffects.Stroke(
        linewidth=lw, foreground='black'), patheffects.Normal()])

def draw_rect(ax, b):
    patch = ax.add_patch(patches.Rectangle(b[:2], *b[-2:], fill=False, edgecolor='white', lw=2))
    draw_outline(patch, 4)

def draw_text(ax, xy, txt, sz=12, color='white', valign='top'):
    text = ax.text(*xy, txt,
                   verticalalignment=valign, color=color, fontsize=sz)  # , weight='bold')
    draw_outline(text, 1)

## end